<a href="https://colab.research.google.com/github/Mahesh-Alyana/oneAPI-Intel/blob/varun/oneAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Load dataset
df = pd.read_csv("/content/training_data_sentiment.csv")

# Remove unnecessary columns
df = df.drop(['Source', 'PublishDate'], axis=1)

# Remove missing values
df = df.dropna()

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens to form text again
    text = " ".join(tokens)
    
    return text

# Preprocess text in the dataset
df['clean_title'] = df['Title'].apply(preprocess_text)


In [67]:
df['clean_headline'] = df['Headline'].apply(preprocess_text)

In [68]:
import gensim

# Convert preprocessed text to gensim dictionary and corpus
texts = [doc.split() for doc in df['clean_title']]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define number of topics and run LDA model
num_topics = 4
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print top words in each topic
for topic in lda_model.show_topics(num_topics=num_topics):
    print(topic)


(0, '0.098*"microsoft" + 0.016*"window" + 0.015*"microsofts" + 0.013*"10" + 0.011*"surface" + 0.010*"new" + 0.009*"’" + 0.007*"one" + 0.006*"cloud" + 0.006*"lumia"')
(1, '0.107*"economy" + 0.018*"china" + 0.018*"economic" + 0.015*"global" + 0.014*"u" + 0.012*"palestinian" + 0.009*"say" + 0.008*"market" + 0.008*"growth" + 0.008*"2016"')
(2, '0.130*"obama" + 0.029*"obamas" + 0.020*"president" + 0.013*"trump" + 0.008*"say" + 0.007*"budget" + 0.006*"gun" + 0.006*"barack" + 0.006*"plan" + 0.006*"gop"')
(3, '0.055*"economy" + 0.028*"palestine" + 0.013*"court" + 0.012*"state" + 0.010*"oil" + 0.008*"u" + 0.008*"supreme" + 0.007*"israel" + 0.006*"house" + 0.006*"white"')


In [69]:
import gensim

# Convert preprocessed text to gensim dictionary and corpus
texts = [doc.split() for doc in df['clean_headline']]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define number of topics and run LDA model
num_topics = 5
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print top words in each topic
for topic in lda_model.show_topics(num_topics=num_topics):
    print(topic)

(0, '0.058*"economy" + 0.018*"economic" + 0.014*"year" + 0.010*"u" + 0.009*"growth" + 0.009*"global" + 0.009*"market" + 0.008*"china" + 0.008*"said" + 0.007*"percent"')
(1, '0.069*"obama" + 0.062*"president" + 0.035*"barack" + 0.013*"u" + 0.011*"said" + 0.011*"obamas" + 0.010*"state" + 0.009*"republican" + 0.009*"house" + 0.009*"washington"')
(2, '0.055*"microsoft" + 0.015*"new" + 0.012*"window" + 0.010*"company" + 0.009*"10" + 0.008*"microsofts" + 0.007*"surface" + 0.006*"one" + 0.006*"announced" + 0.006*"’"')
(3, '0.025*"palestine" + 0.020*"palestinian" + 0.008*"israel" + 0.007*"israeli" + 0.005*"east" + 0.005*"2016" + 0.005*"west" + 0.004*"democratic" + 0.004*"vote" + 0.004*"middle"')
(4, '0.019*"minister" + 0.010*"said" + 0.008*"prime" + 0.007*"chief" + 0.006*"new" + 0.006*"government" + 0.006*"economic" + 0.006*"proposal" + 0.005*"state" + 0.005*"meeting"')


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_title'], df['SentimentTitle'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)




In [73]:
print(y_train.value_counts())


 0.000000    6645
 0.041667     409
 0.044194     368
-0.041667     299
 0.039528     283
             ... 
-0.036182       1
 0.027862       1
-0.091213       1
 0.073546       1
-0.173529       1
Name: SentimentTitle, Length: 6601, dtype: int64


In [74]:
import pandas as pd

# Convert continuous values to discrete labels
y_train = pd.cut(y_train, bins=[-float("inf"),0, float("inf")], labels=[0, 1])
y_test = pd.cut(y_test, bins=[-float("inf"), 0, float("inf")], labels=[0, 1])

In [75]:
print(y_train.value_counts())


0    19814
1    12592
Name: SentimentTitle, dtype: int64


In [76]:
# Train SVM classifier
svm = LinearSVC()
svm.fit(X_train_vec, y_train)

# Predict on test data
y_pred = svm.predict(X_test_vec)
y_pred = pd.cut(y_pred, bins=[-float("inf"), 0, float("inf")], labels=[0, 1])

# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      4921
           1       0.76      0.71      0.73      3181

    accuracy                           0.80      8102
   macro avg       0.79      0.78      0.79      8102
weighted avg       0.80      0.80      0.80      8102



In [77]:
y_pred.value_counts()

0    5166
1    2936
dtype: int64

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_headline'], df['SentimentHeadline'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [79]:
import pandas as pd

# Convert continuous values to discrete labels
y_train = pd.cut(y_train, bins=[-float("inf"),0, float("inf")], labels=[0, 1])
y_test = pd.cut(y_test, bins=[-float("inf"), 0, float("inf")], labels=[0, 1])

In [80]:
# Train SVM classifier
svm = LinearSVC()
svm.fit(X_train_vec, y_train)

# Predict on test data
y_pred = svm.predict(X_test_vec)
y_pred = pd.cut(y_pred, bins=[-float("inf"), 0, float("inf")], labels=[0, 1])

# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      4853
           1       0.74      0.68      0.71      3249

    accuracy                           0.77      8102
   macro avg       0.77      0.76      0.76      8102
weighted avg       0.77      0.77      0.77      8102



In [81]:
y_pred.value_counts()

0    5114
1    2988
dtype: int64